In [1]:
import requests
import copy
import csv
import time
import json
import uuid
import pprint
import asyncio
from typing import List
from urllib import parse
from urllib.parse import quote
import inspect
import datetime

In [2]:
class magic_fstring_function:
    def __init__(self, payload):
        self.payload = payload
        self.cached = None
    def __str__(self):
        if self.cached is None:
            vars = inspect.currentframe().f_back.f_globals.copy()
            vars.update(inspect.currentframe().f_back.f_locals)
            self.cached = self.payload.format(**vars)
        return self.cached

In [3]:
uuid = str(uuid.uuid4())
item_list_api = "https://apis.zigbang.com/v2/items"
describe_list_api = item_list_api + '/list'
item_describe_api = "https://apis.zigbang.com/v3/items?item_ids={item_id}&detail=true"
item_view_url = "https://zigbang.com/home/villa/items/{item_id}"

referer = "https://zigbang.com/home/villa/subways/414/items"
        
headers = {
    'Host':'apis.zigbang.com',
    'Connection':'keep-alive',
    'Pragma':'no-cache',
    'Cache-Control':'no-cache',
    'Accept':'application/json, text/plain, */*',
    'Origin':'https://zigbang.com',
    'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Mobile Safari/537.36',
    'DNT':'1',
    'Sec-Fetch-Site':'same-site',
    'Sec-Fetch-Mode':'cors',
    'Referer':'https://zigbang.com/home/villa/subways/414/items',
    'Accept-Encoding':'gzip, deflate, br',
    'Accept-Language':'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
}

items = {}

In [4]:
def export_items(items):
    fieldnames = [
        'address1',
        'address2',
        'address3',
        'agent_address1',
        'agent_email',
        'agent_lat',
        'agent_lng',
        'agent_mobile',
        'agent_name',
        'agent_phone',
        'building_type',
        'deposit',
        'rent',
        'description',
        'elevator',
        'floor',
        'floor_all',
        'images',
        'local1',
        'local2',
        'local3',
        'manage_cost',
        'manage_cost_inc',
        'movein_date',
        'near_subways',
        'options',
        'parking',
        'pets_text',
        'random_location',
        'room_direction_text',
        'room_type',
        'sales_type',
        'service_type',
        'size',
        'size_m2',
        'title',
        'updated_at2',
        'user_intro',
        'user_name',
        'url'
    ]
    
    filtered_item = [] 

    for item in items:
        item = item['item']
        item_id = item['id']
        url = magic_fstring_function(item_view_url)
        item['url'] = str(url)
        new_item = {}
        for fieldname in fieldnames:
            new_item[fieldname] = item[fieldname]
        filtered_item.append(new_item)
    return filtered_item

In [5]:
def describe_room_list(items):
    item_ids = items
    max_idx = int(len(item_ids) / 30)
    if len(item_ids) % 30 != 0:
        max_idx += 1
    
    items = []
    headers['Accept'] = 'application/json'
    headers['Referer'] = 'https://zigbang.com/home/villa/subways/37/items'
    
    for i in range(max_idx):
        start_idx = i * 30
        end_idx = min((i+1)*30, len(item_ids))
        sub_ids = item_ids[start_idx:end_idx-1]
        item_id = str(sub_ids).replace(' ', '').replace('\'','')
        api = magic_fstring_function(item_describe_api)
        g = api
        resp = requests.get(
            url=str(api),
            headers=headers,
            timeout=5,
        )
        resp.encoding = 'utf-8'
        result = resp.json()
        resp.close()
        items += result['items']
        time.sleep(1)
        
    return items

In [6]:
def get_room_list(
    items,
    domain: str = "zigbang",
    floor_in: str = "ground",
    geohash: str = "wyd",
    rent_gteq: int = 0,
):
    resp = requests.get(
        url=item_list_api,
        params={
            "domain": domain,
            "floor_in": floor_in,
            "geohash": geohash,
            "rent_gteq": rent_gteq,
        },
        headers=headers,
        timeout=5,
    )
    new_items = json.loads(resp.content.decode('utf-8'))
    resp.close()

    for section in new_items['sections']:
        for item in section['item_ids']:
            items.append(str(item))

    return items

In [7]:
items = []

In [8]:
geos_seoul = [
    'wydm',
    'wydq4',
    'wydq5',
    'wydjx',
    'wydjr',
    'wydjq'
] 

for geo in geos_seoul:
    items = get_room_list(items, geohash=geo)
    time.sleep(1)

In [9]:
len(items)

9506

In [10]:
import random

random_items = random.sample(items, 5000)

In [11]:
result = describe_room_list(random_items)
len(result)

4832

In [12]:
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(result[0]['item'])

{ '_floor': '3층',
  'address1': '서울시 서초구 서초동',
  'address2': '1565-4',
  'address3': None,
  'agent_address1': '서울특별시 서초구 동광로18길 25 1층 103호(방배동, 방배엘리제 B동)',
  'agent_comment': '',
  'agent_email': 'hnahyeon@naver.com',
  'agent_lat': 37.4908027648926,
  'agent_lng': 126.990196228027,
  'agent_local1': '서울시',
  'agent_local2': '서초구',
  'agent_mobile': '010-3767-7166',
  'agent_name': '친절한부동산중개사사무소(황나현)',
  'agent_no': 0,
  'agent_phone': '02-535-4988',
  'bjd_code': '',
  'bonbun_code': '1565',
  'bubun_code': '4',
  'building': None,
  'building_type': '-',
  'contract': '서울특별시',
  'deposit': 27000,
  'description': '★ 서초동 예쁜 신축급 투룸 전세 입니다.\r\n'
                 '\r\n'
                 '★ 교대역 역세권으로 도보 5분 거리입니다.\r\n'
                 '\r\n'
                 '★ 융자 없는 안전한 집으로 전세자금대출 가능합니다.\r\n'
                 '\r\n'
                 '★직접 보시면 마음에 드실 좋은 방이에요!\r\n'
                 '\n'
                 '21.11.08에 직방에 등록된 매물입니다.',
  'description_og': '★ 서초동 예쁜 신축급 투룸 전세 입니다.\r\n'
         

In [13]:
filtered_result = export_items(result)
len(filtered_result)

4832

In [14]:
def get_string_image(item):
    image = ''
    
    for list in item:
        image += list['url'] + '?w=400&h=300&q=70&a=1,'
        
    return image[:-1]

In [15]:
def get_full_address(item):
    addr = item['address1']
    addr += ' '
    addr += item['address2'] or ''
    addr += ' '
    addr += item['address3'] or ''
    return addr

In [16]:
def get_location_lat(item):
    locationList = item['random_location'].split(',')

    lat = locationList[0][0:13]
    
    return lat

def get_location_lng(item):
    locationList = item['random_location'].split(',')

    lng = locationList[1][0:14]
    
    return lng

In [17]:
def get_agent_lat(item):
    agent_lat = str(item['agent_lat'])[0:13]
    
    return agent_lat

def get_agent_lng(item):
    agent_lng = str(item['agent_lng'])[0:14]
    
    return agent_lng

In [18]:
for item in filtered_result:
    item['address'] = get_full_address(item)
    item['lat'] = get_location_lat(item)
    item['lng'] = get_location_lng(item)
    item['image'] = get_string_image(item['images'])
    item['agent_lat'] = get_agent_lat(item)
    item['agent_lng'] = get_agent_lng(item)    

In [19]:
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(filtered_result[0])

{ 'address': '서울시 서초구 서초동 1565-4 ',
  'address1': '서울시 서초구 서초동',
  'address2': '1565-4',
  'address3': None,
  'agent_address1': '서울특별시 서초구 동광로18길 25 1층 103호(방배동, 방배엘리제 B동)',
  'agent_email': 'hnahyeon@naver.com',
  'agent_lat': '37.4908027648',
  'agent_lng': '126.9901962280',
  'agent_mobile': '010-3767-7166',
  'agent_name': '친절한부동산중개사사무소(황나현)',
  'agent_phone': '02-535-4988',
  'building_type': '-',
  'deposit': 27000,
  'description': '★ 서초동 예쁜 신축급 투룸 전세 입니다.\r\n'
                 '\r\n'
                 '★ 교대역 역세권으로 도보 5분 거리입니다.\r\n'
                 '\r\n'
                 '★ 융자 없는 안전한 집으로 전세자금대출 가능합니다.\r\n'
                 '\r\n'
                 '★직접 보시면 마음에 드실 좋은 방이에요!\r\n'
                 '\n'
                 '21.11.08에 직방에 등록된 매물입니다.',
  'elevator': '있음',
  'floor': '3층',
  'floor_all': '5층',
  'image': 'https://ic.zigbang.com/ic/items/29364848/1.jpg?w=400&h=300&q=70&a=1,https://ic.zigbang.com/ic/items/29364848/2.jpg?w=400&h=300&q=70&a=1,https://ic.zigbang.com/ic/items/2

In [20]:
with open('villa.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = [
        'address',
        'agent_address1',
        'agent_email',
        'agent_lat',
        'agent_lng',
        'agent_mobile',
        'agent_name',
        'agent_phone',
        'building_type',
        'deposit',
        'rent',
        'description',
        'elevator',
        'floor',
        'floor_all',
        'image',
        'lat',
        'lng',
        'local1',
        'local2',
        'local3',
        'manage_cost',
        'manage_cost_inc',
        'movein_date',
        'near_subways',
        'options',
        'parking',
        'pets_text',
        'room_direction_text',
        'room_type',
        'sales_type',
        'service_type',
        'size',
        'size_m2',
        'title',
        'updated_at2',
        'user_intro',
        'user_name',
        'url'
    ]
    writer = csv.DictWriter(csvfile, extrasaction='ignore', fieldnames=fieldnames)
    writer.writeheader()
    for item in filtered_result:
        writer.writerow(item)

In [1]:
# 크롤링한 csv 파일까지 커밋했으니까 이것만 run 하시면 됩니다!!!

import pymysql
import csv
conn = pymysql.connect(host='localhost', user='khweb', password='khWeb@123', db='mybang_db', charset='utf8mb4')
curs = conn.cursor()

sql = """insert into villa (address, agent_address, agent_email, agent_lat, agent_lng,
                            agent_mobile, agent_name, agent_phone, building_type, deposit, rent,
                            description, elevator, floor, floor_all, image,
                            lat, lng, local1, local2, local3,
                            manage_cost, manage_cost_inc, movein_date, near_subways, options,
                            parking, pets, room_direction, room_type, sales_type,
                            service_type, size, size_m2, title, updated_at,
                            user_intro, user_name, url)
                            values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                                    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                                    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                                    %s,%s,%s,%s,%s,%s,%s,%s,%s)"""

f = open('villa.csv', 'r', encoding='utf-8')
rd = csv.reader(f)

next(rd)

for line in rd:
    curs.execute(sql, (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], 
                       line[11], line[12], line[13], line[14], line[15], line[16], line[17], line[18], line[19], line[20], 
                       line[21], line[22], line[23], line[24], line[25], line[26], line[27], line[28], line[29], line[30], 
                       line[31], line[32], line[33], line[34], line[35], line[36], line[37], line[38]))
conn.commit()
conn.close()
f.close()